# Modeling Agricultural Variables
## Python modules

In [1]:
import warnings
import time
import os

import dask
from dask.distributed import Client

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import geopandas as gpd

import pyarrow
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr
from scipy.linalg import LinAlgWarning
from scipy.stats import pearsonr

import math
import seaborn as sns
import random

# Mild Preprocessing
## 1. Load in the Data

### Read in the features
First, we load in the feature data. This data was aggregated in the ___ notebook

In [2]:
# Let's read in the new concatenated features:
#features = pd.read_feather("/capstone/mosaiks/repos/modeling/data/cropmosaiks_features_landsat8.feather")
#features = pd.read_feather("/capstone/mosaiks/repos/modeling/data/features_sea_save.feather")
features = gpd.read_feather("/capstone/mosaiks/repos/modeling/data/sentinel_rgb_features_sea_save_2023_04_24.feather")


# We can remove year 2019 since it is unfortunately bunk
#features = features.drop(features[features['year'] = 2019].index, inplace = True)
#features = features[features.year != 2019]
#features = features[features.year != 2017]
print(pd.unique(features['year']))
features

[2015 2016 2017 2018 2019 2020 2021 2022]


,0,1,2,3,4,5,6,7,8,9,...,995,996,997,998,999,lon,lat,year,month,geometry
0,0.001058,0.000000,0.005181,1.014728,0.098902,0.000000,0.033511,1.810280,0.0,0.000000,...,3.478022,4.774719,0.011104,0.840888,0.000045,27.47466,-16.339357,2015,7,POINT (27.47466 -16.33936)
1,0.002010,0.000022,0.003418,1.058190,0.125574,0.000291,0.039863,1.858072,0.0,0.000029,...,3.526972,4.882292,0.024313,0.873838,0.000047,27.46466,-16.339357,2015,7,POINT (27.46466 -16.33936)
2,0.004124,0.000000,0.000000,1.069269,0.127892,0.000000,0.036240,1.871935,0.0,0.000054,...,3.552448,4.921965,0.022708,0.881511,0.000000,27.46466,-16.329357,2015,7,POINT (27.46466 -16.32936)
3,0.001559,0.000004,0.000000,1.103106,0.134511,0.000230,0.057822,1.941573,0.0,0.000000,...,3.571980,4.965928,0.021992,0.857326,0.000266,27.45466,-16.339357,2015,7,POINT (27.45466 -16.33936)
4,0.001731,0.000002,0.005510,1.056985,0.122743,0.000167,0.034343,1.860240,0.0,0.000000,...,3.535786,4.864718,0.009189,0.852267,0.000000,27.47466,-16.349357,2015,7,POINT (27.47466 -16.34936)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3067,0.004385,0.000510,0.008147,1.396553,0.238291,0.000534,0.236429,2.404210,0.0,0.000024,...,3.850011,5.434838,0.012184,0.761620,0.000007,27.96466,-16.739357,2022,12,POINT (27.96466 -16.73936)
3068,0.000493,0.001652,0.002549,1.860167,0.405460,0.001686,0.382340,3.069240,0.0,0.000000,...,4.408813,6.329021,0.021479,0.742347,0.000000,27.97466,-16.739357,2022,12,POINT (27.97466 -16.73936)
3069,0.006834,0.000185,0.006003,1.653710,0.388718,0.000715,0.292500,2.731598,0.0,0.000031,...,4.198204,5.937481,0.013647,0.769534,0.000002,27.97466,-16.749357,2022,12,POINT (27.97466 -16.74936)
3070,0.003454,0.000005,0.001523,1.121913,0.219986,0.000052,0.136004,1.987867,0.0,0.000000,...,3.586843,4.973966,0.024207,0.821185,0.000817,27.97466,-16.759357,2022,12,POINT (27.97466 -16.75936)


In [3]:
# Extract the size of the features dataframe
rows, cols = features.shape

# compute the number of feature cells in the features dataframe
num_cells = rows * (cols-3)
num_cells

# Let's check how many rows do not have NA values:
len(features.dropna()) # 56638
len(features)
# This makes sense that all rows contain at least 1 NA value 
# since the features we currently use were randomly sampled from 10% of the points in Zambia.
nan_count = features.isna().sum().sum()
print(nan_count)
print(rows)
print(cols)
print(num_cells)

0
549303
1005
550401606


### Read in Ground-Truth Data
Next, we read in our ground truth data, which was processed in the Preprocessing notebook. Make sure to run that if you are getting any errors from running this code.

In [4]:
names = ["sea_unq", "year", "total_area_planted_ha", "total_area_harv_ha", "total_area_lost_ha", "total_harv_kg", 
         "yield_kgha", "frac_area_harv", "frac_area_loss", "area_lost_fire", "maize", "groundnuts", "mixed_beans", 
         "popcorn", "sorghum", "soybeans", "sweet_potatoes", "bunding", "monocrop", "mixture", "frac_loss_drought", 
         "frac_loss_flood", "frac_loss_animal", "frac_loss_pests", "frac_loss_soil", "frac_loss_fert", "prop_till_plough", 
         "prop_till_ridge", "prop_notill", "prop_hand", "prop_mono", "prop_mix", "log_maize", 
         "log_sweetpotatoes", "log_groundnuts", "log_soybeans", "loss_ind", "drought_loss_ind", "flood_loss_ind", 
         "animal_loss_ind", "pest_loss_ind", "geometry"]


#country_sea = gpd.read_file('/capstone/mosaiks/repos/preprocessing/ground_data_clean/total3.shp', encoding = "utf-8")
#country_sea = gpd.read_file('/capstone/mosaiks/repos/preprocessing/ground_data_clean/updated_data.shp')
country_sea = gpd.read_file('/capstone/mosaiks/repos/preprocessing/data/ground_data_spatial/updated_data.shp')

country_sea = country_sea[country_sea.year != 2019]
#country_sea = country_sea[country_sea.year != 2017]
print(pd.unique(country_sea['year']))
country_sea.columns = names
country_sea.info

ERROR 1: PROJ: proj_create_from_database: Open of /Users/hveirs/.conda/envs/mosaiks/share/proj failed


[2009. 2010. 2011. 2012. 2013. 2014. 2016. 2017. 2020. 2021.]


<bound method DataFrame.info of       sea_unq    year  total_area_planted_ha  total_area_harv_ha  \
0           1  2009.0                34.9725           34.972500   
1           1  2010.0                32.2150           26.762500   
2           1  2011.0                60.4075           59.752500   
3           1  2012.0                64.6175           57.602500   
4           1  2013.0               325.5000          282.750000   
...       ...     ...                    ...                 ...   
3600      392  2014.0               485.6942          116.705696   
3601      392  2016.0              5193.0000         3272.000000   
3602      392  2017.0              2652.7500         1261.000000   
3603      392  2020.0               103.5250           66.501000   
3604      392  2021.0                41.6625           32.410000   

      total_area_lost_ha  total_harv_kg   yield_kgha  frac_area_harv  \
0               0.000000        57563.0  1645.950390        1.000000   
1      

In [5]:
# Extract the size of the features dataframe
rows, cols = country_sea.shape

# compute the number of feature cells in the features dataframe
num_cells = rows * (cols-3)
num_cells

# Let's check how many rows do not have NA values:
len(country_sea.dropna()) # 56638
len(country_sea)
# This makes sense that all rows contain at least 1 NA value 
# since the features we currently use were randomly sampled from 10% of the points in Zambia.
nan_count = country_sea.isna().sum().sum()
print(nan_count)
print(rows)
print(cols)
print(num_cells)

0
3605
42
140595


We're going to make another object `sea_unq_join` which contains the spatial information and a unique key for each SEA. This will be handy later, when we need to join the features to the ground-truth data.

In [6]:
# Filter country_sea for unique values of 'seq_unq' and 'geometry'
sea_unq_join = country_sea[['sea_unq', 'geometry']].drop_duplicates()

# Display the filtered DataFrame
print(sea_unq_join)


      sea_unq                                           geometry
0           1  POLYGON ((27.82327 -13.65772, 27.82294 -13.657...
10          2  POLYGON ((27.99349 -13.46497, 27.99352 -13.464...
20          3  POLYGON ((28.09909 -13.51864, 28.09867 -13.516...
29          4  POLYGON ((28.31924 -13.42915, 28.31911 -13.426...
38          5  POLYGON ((28.39982 -13.51544, 28.40012 -13.514...
...       ...                                                ...
3571      388  POLYGON ((25.07771 -14.63920, 25.07732 -14.638...
3578      389  POLYGON ((22.74142 -14.00343, 22.73856 -14.002...
3585      390  POLYGON ((23.08604 -14.20026, 23.08957 -14.202...
3592      391  POLYGON ((24.36764 -16.62208, 24.36564 -16.621...
3599      392  POLYGON ((23.23962 -16.31204, 23.23876 -16.312...

[392 rows x 2 columns]


## 2. Organize the features by growing season



In [7]:
# Organize the features by growing season
# Carry months October, November, and December over to the following year's data
# These months represent the start of the growing season for the following year's maize yield
year_end = 2023

features['year'] = np.where(
    features['month'].isin([10, 11, 12]),
    features['year'] + 1, 
    features['year'])

features_gs = features[features['year'] <= year_end]

features_gs.sort_values(['year', 'month'], inplace=True)

Look at whether or not the imputation benefits from only using the growing season months

## 3. Pivot Wider by months

Since we want each row to represent one location per year, we can use the .unstack() function to pivot wider all rows with the same lat/lon and year. This results in a dataframe with 12,000 columns (1,000 columns for each month). 

In [8]:
# Use the unstack() function to pivot wider the rows with the same lat/lon 
features = features_gs.set_index(['lon', 'lat', 'year', 'month']).unstack()

# Apply a transformation to the columns' names
features.columns = features.columns.map(lambda x: '{}_{}'.format(*x))

In [9]:
# Since our features have infinite values, it is important to replace those with NaN values.
features.replace([np.inf, -np.inf], np.nan, inplace=True)
features = features.reset_index()
features.iloc[:, 12000:]

,999_10,999_11,999_12,geometry_1,geometry_2,geometry_3,geometry_4,geometry_5,geometry_6,geometry_7,geometry_8,geometry_9,geometry_10,geometry_11,geometry_12
0,NaN,NaN,NaN,None,None,None,None,None,None,None,None,POINT (22.00466 -16.18936),None,None,None
1,0.000102,0.000338,0.000273,None,POINT (22.00466 -16.18936),None,POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936)
2,0.000039,0.021058,NaN,None,None,None,POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),None
3,0.000114,0.001221,NaN,POINT (22.00466 -16.18936),None,None,POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),None
4,0.000064,0.000242,0.000298,POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72513,0.000004,0.000041,0.000078,POINT (33.50466 -10.20936),None,None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936)
72514,0.000015,0.000032,0.000130,None,None,None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936)
72515,0.000021,0.000219,NaN,None,None,POINT (33.50466 -10.20936),None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),None
72516,NaN,0.000204,0.000255,None,None,None,None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936)


### 4. Convert the features into a Geo Dataframe

This step allows us to join the features with our clean, ground-truth survey data based on the geometries.

In [10]:
# Create a geodataframe of the new features
features_gdf = gpd.GeoDataFrame(
    features, # could change to features_gs
    geometry = gpd.points_from_xy(x = features.lon, y = features.lat), 
    crs='EPSG:4326'
)

In [11]:
features_gdf.iloc[:,12000:]

,999_10,999_11,999_12,geometry_1,geometry_2,geometry_3,geometry_4,geometry_5,geometry_6,geometry_7,geometry_8,geometry_9,geometry_10,geometry_11,geometry_12,geometry
0,NaN,NaN,NaN,None,None,None,None,None,None,None,None,POINT (22.00466 -16.18936),None,None,None,POINT (22.00466 -16.18936)
1,0.000102,0.000338,0.000273,None,POINT (22.00466 -16.18936),None,POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936)
2,0.000039,0.021058,NaN,None,None,None,POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),None,POINT (22.00466 -16.18936)
3,0.000114,0.001221,NaN,POINT (22.00466 -16.18936),None,None,POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),None,POINT (22.00466 -16.18936)
4,0.000064,0.000242,0.000298,POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936),POINT (22.00466 -16.18936)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72513,0.000004,0.000041,0.000078,POINT (33.50466 -10.20936),None,None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936)
72514,0.000015,0.000032,0.000130,None,None,None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936)
72515,0.000021,0.000219,NaN,None,None,POINT (33.50466 -10.20936),None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),None,POINT (33.50466 -10.20936)
72516,NaN,0.000204,0.000255,None,None,None,None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),None,POINT (33.50466 -10.20936),POINT (33.50466 -10.20936),POINT (33.50466 -10.20936)


In [12]:
# Drop the redundant independent lon and lat columns because now that they are in a separate geometry column
geo_list = ['geometry_1', 'geometry_2', 'geometry_3', 'geometry_4', 'geometry_5', 'geometry_6', 'geometry_7', 
            'geometry_8', 'geometry_9', 'geometry_10', 'geometry_11', 'geometry_12']
features_gdf = features_gdf.drop(geo_list, axis = 1)

features_gdf.iloc[:, 12000:]
features_gdf.geometry

0        POINT (22.00466 -16.18936)
1        POINT (22.00466 -16.18936)
2        POINT (22.00466 -16.18936)
3        POINT (22.00466 -16.18936)
4        POINT (22.00466 -16.18936)
                    ...            
72513    POINT (33.50466 -10.20936)
72514    POINT (33.50466 -10.20936)
72515    POINT (33.50466 -10.20936)
72516    POINT (33.50466 -10.20936)
72517    POINT (33.50466 -10.20936)
Name: geometry, Length: 72518, dtype: geometry

In [13]:
features_gdf.iloc[:,:]

,lon,lat,year,0_1,0_2,0_3,0_4,0_5,0_6,0_7,...,999_4,999_5,999_6,999_7,999_8,999_9,999_10,999_11,999_12,geometry
0,22.00466,-16.189357,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000210,NaN,NaN,NaN,POINT (22.00466 -16.18936)
1,22.00466,-16.189357,2016,NaN,0.000038,NaN,0.000000,0.000000e+00,5.606620e-05,0.000000,...,1.000000,1.000000,0.000197,1.000000,0.000144,0.000061,0.000102,0.000338,0.000273,POINT (22.00466 -16.18936)
2,22.00466,-16.189357,2017,NaN,NaN,NaN,0.000000,3.640897e-07,8.515847e-06,0.000068,...,0.004659,0.001663,0.001083,0.000475,0.000215,0.000102,0.000039,0.021058,NaN,POINT (22.00466 -16.18936)
3,22.00466,-16.189357,2018,0.000032,NaN,NaN,0.000000,0.000000e+00,9.116117e-07,0.000044,...,0.003729,0.002599,0.001340,0.000328,0.000404,0.000182,0.000114,0.001221,NaN,POINT (22.00466 -16.18936)
4,22.00466,-16.189357,2019,0.000080,0.000025,0.000026,0.000078,3.502390e-05,1.300048e-04,0.000079,...,0.001003,0.000488,0.000445,0.000310,0.000058,0.000035,0.000064,0.000242,0.000298,POINT (22.00466 -16.18936)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72513,33.50466,-10.209357,2019,0.005556,NaN,NaN,0.000067,4.927176e-05,5.067072e-05,0.000145,...,0.005145,0.003170,0.002521,0.000776,0.000474,0.000011,0.000004,0.000041,0.000078,POINT (33.50466 -10.20936)
72514,33.50466,-10.209357,2020,NaN,NaN,NaN,0.000080,1.206588e-04,NaN,0.000283,...,0.004205,0.007104,NaN,0.001003,0.000172,0.000003,0.000015,0.000032,0.000130,POINT (33.50466 -10.20936)
72515,33.50466,-10.209357,2021,NaN,NaN,0.000100,NaN,7.305521e-05,1.146089e-04,0.000278,...,NaN,0.004004,0.002973,0.001367,0.000131,0.000033,0.000021,0.000219,NaN,POINT (33.50466 -10.20936)
72516,33.50466,-10.209357,2022,NaN,NaN,NaN,NaN,2.063466e-04,1.241788e-04,0.000497,...,NaN,0.004903,0.003632,0.001904,0.000701,0.000214,NaN,0.000204,0.000255,POINT (33.50466 -10.20936)


## 5. Join features to ground data

This is an important step, since this is how we can use both the features and the data to use in models.

In [14]:
# Now lets combine the sea data 
spatial_join = gpd.sjoin(features_gdf, sea_unq_join, how='right', predicate = 'within')

In [15]:
spatial_join

,index_left,lon,lat,year,0_1,0_2,0_3,0_4,0_5,0_6,...,999_5,999_6,999_7,999_8,999_9,999_10,999_11,999_12,sea_unq,geometry
0,46330.0,27.81466,-13.669357,2017.0,NaN,NaN,0.000000,NaN,0.000000,0.000000,...,0.532948,0.469076,0.007786,0.006779,0.004811,0.001675,0.029891,NaN,1,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
0,46333.0,27.81466,-13.669357,2020.0,NaN,0.000000,NaN,0.000000,0.000000,0.000000,...,0.474246,0.417571,0.135569,0.003355,0.004876,0.003185,0.187867,0.156783,1,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
0,46332.0,27.81466,-13.669357,2019.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.490193,0.392582,0.325692,0.004587,0.002309,0.002191,0.002333,NaN,1,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
0,46334.0,27.81466,-13.669357,2021.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,...,0.645601,0.673485,0.631725,0.004959,0.001359,0.001627,0.052683,NaN,1,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
0,46329.0,27.81466,-13.669357,2016.0,0.000000,NaN,NaN,0.000000,0.000000,0.000000,...,0.679722,1.000000,0.324600,0.006641,0.003481,NaN,NaN,1.000000,1,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3599,7316.0,23.24466,-16.269357,2022.0,NaN,0.003581,0.003867,0.001019,0.001521,0.000000,...,0.009622,0.008655,0.003418,0.001569,0.000234,0.000054,0.000039,0.000276,392,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
3599,7315.0,23.24466,-16.269357,2021.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.002137,...,0.009862,0.002694,0.000432,0.000132,0.000086,0.000063,0.000060,NaN,392,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
3599,7314.0,23.24466,-16.269357,2020.0,0.002399,NaN,0.000000,0.000000,0.001860,0.002130,...,0.013037,0.007480,0.002309,0.000083,0.000194,0.000100,0.000016,0.004205,392,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
3599,7313.0,23.24466,-16.269357,2019.0,0.003386,0.000000,0.000000,0.000000,0.000923,0.002777,...,0.007071,0.001832,0.001039,0.000223,0.000081,0.000324,0.001030,0.004801,392,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."


In [16]:
features_join = spatial_join.merge(country_sea, on=['year', 'sea_unq'], how='inner')

In [17]:
features_join

,index_left,lon,lat,year,0_1,0_2,0_3,0_4,0_5,0_6,...,log_maize,log_sweetpotatoes,log_groundnuts,log_soybeans,loss_ind,drought_loss_ind,flood_loss_ind,animal_loss_ind,pest_loss_ind,geometry_y
0,46330.0,27.81466,-13.669357,2017.0,NaN,NaN,0.000000,NaN,0.000000,0.000000,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
1,46285.0,27.80466,-13.659357,2017.0,NaN,NaN,0.000000,NaN,0.000000,0.000000,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
2,46294.0,27.80466,-13.649357,2017.0,NaN,NaN,0.000030,NaN,0.000000,0.000000,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
3,46333.0,27.81466,-13.669357,2020.0,NaN,0.0,NaN,0.000000,0.000000,0.000000,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
4,46288.0,27.80466,-13.659357,2020.0,NaN,0.0,NaN,0.000000,0.000000,0.000000,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32926,7307.0,23.24466,-16.279357,2020.0,0.001146,NaN,0.000416,0.000353,0.000348,0.000426,...,5.648800,-inf,6.830794,-inf,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32927,7571.0,23.26466,-16.279357,2020.0,0.005164,NaN,0.000000,0.000000,0.000000,0.000172,...,5.648800,-inf,6.830794,-inf,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32928,7197.0,23.23466,-16.279357,2020.0,0.000717,NaN,0.000319,0.000559,0.000502,0.000581,...,5.648800,-inf,6.830794,-inf,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32929,7430.0,23.25466,-16.269357,2020.0,0.004515,NaN,0.000000,0.000000,0.000000,0.000167,...,5.648800,-inf,6.830794,-inf,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."


In [18]:
# Drop the redundant independent lon and lat columns because now that they are in a separate geometry column
#features_join = features_join.drop(['lon', 'lat', 'geometry_x'], axis = 1)
#features_join = features_join.drop(['lon', 'lat'], axis = 1)
features_join = features_join.drop(['geometry_x'], axis = 1)

In [19]:
features_join.iloc[:, 12000:] 

,999_9,999_10,999_11,999_12,sea_unq,total_area_planted_ha,total_area_harv_ha,total_area_lost_ha,total_harv_kg,yield_kgha,...,log_maize,log_sweetpotatoes,log_groundnuts,log_soybeans,loss_ind,drought_loss_ind,flood_loss_ind,animal_loss_ind,pest_loss_ind,geometry_y
0,0.004811,0.001675,0.029891,NaN,1,5329.234136,1662.551496,3666.68264,37750.0,7.083569,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
1,0.000948,0.000931,0.002581,NaN,1,5329.234136,1662.551496,3666.68264,37750.0,7.083569,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
2,0.000402,0.000466,0.000509,NaN,1,5329.234136,1662.551496,3666.68264,37750.0,7.083569,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
3,0.004876,0.003185,0.187867,0.156783,1,75.812500,62.377500,13.43500,139700.0,1842.704040,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
4,0.001853,0.001024,0.002458,0.003444,1,75.812500,62.377500,13.43500,139700.0,1842.704040,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32926,0.001382,0.001083,0.001000,0.004963,392,103.525000,66.501000,37.02400,35620.0,344.071480,...,5.648800,-inf,6.830794,-inf,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32927,0.004126,0.002277,0.002202,0.003774,392,103.525000,66.501000,37.02400,35620.0,344.071480,...,5.648800,-inf,6.830794,-inf,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32928,0.001942,0.001737,0.001547,0.006698,392,103.525000,66.501000,37.02400,35620.0,344.071480,...,5.648800,-inf,6.830794,-inf,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32929,0.002867,0.001590,0.001557,0.003505,392,103.525000,66.501000,37.02400,35620.0,344.071480,...,5.648800,-inf,6.830794,-inf,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."


In [20]:
# Set some parameters 
# Number of features:
num_features = 1000

# Imputing
#impute_manual = False
impute_manual = True

### 6. Impute missing values

Imputing "manually" by descending group levels imputes NA values in multiple "cascading" steps, decreasing the proportion of inputed values with each step. First, the NA values are imputed at by both `year` and `geometry`, which should yield imputed values that most closely match the feature values that would be present in the data if there was no clouds obscuring the satellite images. Next, the remaining NA values that could not be imputed by both `year` and `district` are imputed by only `district`. Lastly, the remaining NA vlaues that could not be imputed by both `year` and `district` or by just `district` are imputed by `year` only. This option gives the user more control and transparency over how the imputation is executed.

Imputing using `scikit learn`'s simple imputer executes standard imputation, the details of which can be found in the `scikitlearn` documentation [here.](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

The imputation approach depends on the selection made at the top of this notebook for `impute_manual`.

In [21]:
# Cropmosaiks used this line to calculate the number of cells, but I don't know what this does. 
#num_cells = len(features) * len(month_range) * num_features

# Extract the size of the features dataframe
rows, cols = features_join.shape

# compute the number of feature cells in the features dataframe
num_cells = rows * cols
num_cells

# Let's check how many rows do not have NA values:
len(features_join.dropna())
# This makes sense that all rows contain at least 1 NA value 
# since the features we currently use were randomly sampled from 10% of the points in Zambia.
nan_count = features_join.isna().sum().sum()
print(nan_count)
print(rows)
print(cols)
print(num_cells)

117699000
32931
12045
396653895


In [22]:
features_join = features_join.replace([np.inf, -np.inf], np.nan)
#np.isinf(features_join).values.sum()
#features_join.isin([np.inf, -np.inf])
features_join

,index_left,lon,lat,year,0_1,0_2,0_3,0_4,0_5,0_6,...,log_maize,log_sweetpotatoes,log_groundnuts,log_soybeans,loss_ind,drought_loss_ind,flood_loss_ind,animal_loss_ind,pest_loss_ind,geometry_y
0,46330.0,27.81466,-13.669357,2017.0,NaN,NaN,0.000000,NaN,0.000000,0.000000,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
1,46285.0,27.80466,-13.659357,2017.0,NaN,NaN,0.000000,NaN,0.000000,0.000000,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
2,46294.0,27.80466,-13.649357,2017.0,NaN,NaN,0.000030,NaN,0.000000,0.000000,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
3,46333.0,27.81466,-13.669357,2020.0,NaN,0.0,NaN,0.000000,0.000000,0.000000,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
4,46288.0,27.80466,-13.659357,2020.0,NaN,0.0,NaN,0.000000,0.000000,0.000000,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32926,7307.0,23.24466,-16.279357,2020.0,0.001146,NaN,0.000416,0.000353,0.000348,0.000426,...,5.648800,NaN,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32927,7571.0,23.26466,-16.279357,2020.0,0.005164,NaN,0.000000,0.000000,0.000000,0.000172,...,5.648800,NaN,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32928,7197.0,23.23466,-16.279357,2020.0,0.000717,NaN,0.000319,0.000559,0.000502,0.000581,...,5.648800,NaN,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32929,7430.0,23.25466,-16.269357,2020.0,0.004515,NaN,0.000000,0.000000,0.000000,0.000167,...,5.648800,NaN,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."


In [23]:
# This code chunk will help us keep track of the imputation process by defining colors for the messages.
class bcolors:
    BL = '\x1b[1;34m' #GREEN
    GR = '\x1b[1;36m' #GREEN
    YL = '\x1b[1;33m' #YELLOW
    RD = '\x1b[1;31m' #RED
    RESET = '\033[0m' #RESET COLOR

In [24]:
#impute_median = True
impute_median = False

In [25]:
# Carlo's imputation

if impute_median: # Replace all infinity values with NaN
    features_join = features_join.replace([np.inf, -np.inf], np.nan)
    features_join = (
        features_join
        .fillna(features_join
                .groupby(['lat', 'lon'], as_index=False)
                .transform('median')
                )
    )
    features_join = (
        features_join
        .fillna(features_join
                .groupby(['sea_unq'], as_index=False)
                .transform('median')
                )
    )
else: 
    impute_manual = True

#### Notes:
Using cropmosaiks' features from 2022 has a starting row count of 1893, and an ending total row count of 325. 
This might be due to the lack of temporal overlap between those features and our new survey data (the survey data we use currently only covers 2 years).

In [26]:
features_join.iloc[:, 505:]
#features_join.iloc[:, 11995:]

,41_10,41_11,41_12,42_1,42_2,42_3,42_4,42_5,42_6,42_7,...,log_maize,log_sweetpotatoes,log_groundnuts,log_soybeans,loss_ind,drought_loss_ind,flood_loss_ind,animal_loss_ind,pest_loss_ind,geometry_y
0,1.141906,0.968697,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
1,1.340785,1.161160,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
2,1.194933,1.089161,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
3,0.591394,0.325480,0.090580,NaN,0.0,NaN,0.0,0.0,0.0,0.0,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
4,1.315837,1.325033,0.645732,NaN,0.0,NaN,0.0,0.0,0.0,0.0,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32926,2.595842,3.081010,1.214092,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,5.648800,NaN,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32927,1.530862,1.628284,0.949406,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,5.648800,NaN,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32928,2.576555,2.952143,0.803103,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,5.648800,NaN,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
32929,1.714823,2.134003,1.006237,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,5.648800,NaN,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."


In [27]:
# for col in features_join.columns:
#     col_str = str(col)
#     if col_str in group_names:
#         split_col = col_str.split("_")
#         # if len(split_col) == 2:
#         #     feature, month = split_col
#         #     assert int(feature) in range(1000)
#         #     assert int(month) in range(1, 13)
#         for group in groups:
#             features_join[group] = features_join[group].fillna(method="bfill", axis=1)
#             print(f"Filled NAs for {group}")

In [28]:
features_test = features_join.iloc[:1000, :]

In [35]:
# First, save the features groups by the column name 
#(i.e. we want only feature 1 values to be imputed for feature 1)
group_names = [f"{i}_{j}" for i in range(1000) for j in range(1, 13)]
groups = [group_names[i:i+12] for i in range(0, len(group_names), 12)]

# Then use a for loop to basically make sure the columns are in the feature form. 
# We don't want to impute features based on survey columns.
last_group_processed = False
for col in features_test.columns:
    col_str = str(col)
    if col_str in group_names:
        split_col = col_str.split("_")
        for group in groups:
            features_test[group] = features_test[group].fillna(method="bfill", axis=1)
            print(f"Filled NAs for {group}")
            if group == groups[-1]:
                last_group_processed = True
        if last_group_processed:
            break


/tmp/ipykernel_3027019/265155464.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_test[group] = features_test[group].fillna(method="bfill", axis=1)


Filled NAs for ['0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8', '0_9', '0_10', '0_11', '0_12']


/tmp/ipykernel_3027019/265155464.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_test[group] = features_test[group].fillna(method="bfill", axis=1)


Filled NAs for ['1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8', '1_9', '1_10', '1_11', '1_12']


/tmp/ipykernel_3027019/265155464.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_test[group] = features_test[group].fillna(method="bfill", axis=1)


...

Filled NAs for ['997_1', '997_2', '997_3', '997_4', '997_5', '997_6', '997_7', '997_8', '997_9', '997_10', '997_11', '997_12']


/tmp/ipykernel_3027019/265155464.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_test[group] = features_test[group].fillna(method="bfill", axis=1)


Filled NAs for ['998_1', '998_2', '998_3', '998_4', '998_5', '998_6', '998_7', '998_8', '998_9', '998_10', '998_11', '998_12']


/tmp/ipykernel_3027019/265155464.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_test[group] = features_test[group].fillna(method="bfill", axis=1)


Filled NAs for ['999_1', '999_2', '999_3', '999_4', '999_5', '999_6', '999_7', '999_8', '999_9', '999_10', '999_11', '999_12']


# ---- Impute Manual Bfill technique ----

In [29]:
features_join.iloc[:1000, 505:755]
#features_join

,41_10,41_11,41_12,42_1,42_2,42_3,42_4,42_5,42_6,42_7,...,61_10,61_11,61_12,62_1,62_2,62_3,62_4,62_5,62_6,62_7
0,1.141906,0.968697,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,1.595295,1.352799,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0
1,1.340785,1.161160,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,1.347891,1.097611,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0
2,1.194933,1.089161,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,1.099367,1.023362,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0
3,0.591394,0.325480,0.090580,NaN,0.0,NaN,0.0,0.0,0.0,0.0,...,1.080549,0.488113,0.219602,NaN,0.0,NaN,0.0,0.0,0.0,0.0
4,1.315837,1.325033,0.645732,NaN,0.0,NaN,0.0,0.0,0.0,0.0,...,1.304484,1.365465,1.089622,NaN,0.0,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.819797,0.334932,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,1.081678,0.721561,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
996,1.203353,0.707047,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,1.465989,1.099513,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
997,0.678997,0.374356,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.999881,0.751045,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
998,0.905363,0.675709,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,1.147266,0.986570,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [30]:
if impute_manual:
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Post step 1 NaN row count: {bcolors.YL}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPost step 1 NaN row %: {bcolors.YL}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPost step 1 NaN cell %: {bcolors.YL}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 2: Filling NaN values by year and SEA across group average')
    # First, save the features groups by the column name 
    #(i.e. we want only feature 1 values to be imputed for feature 1)
    group_names = [f"{i}_{j}" for i in range(1000) for j in range(1, 13)]
    groups = [group_names[i:i+12] for i in range(0, len(group_names), 12)]

    # Then use a for loop to basically make sure the columns are in the feature form. 
    # We don't want to impute features based on survey columns.
    last_group_processed = False
    for col in features_join.columns:
        col_str = str(col)
        if col_str in group_names:
            split_col = col_str.split("_")
            for group in groups:
                features_join[group] = features_join[group].fillna(method="bfill", axis=1)
                print(f"Filled NAs for {group}")
                if group == groups[-1]:
                    last_group_processed = True
            if last_group_processed:
                break
                
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Post step 1 NaN row count: {bcolors.YL}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPost step 1 NaN row %: {bcolors.YL}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPost step 1 NaN cell %: {bcolors.YL}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 2: Filling NaN values by year and SEA across group average')
    features_join = (
        features_join
        .fillna(features_join
                .groupby(['year', 'sea_unq'], as_index=False) 
                .transform('median')
               )
    )
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Post step 2 NaN row count: {bcolors.YL}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPost step 2 NaN row %: {bcolors.YL}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPost step 2 NaN cell %: {bcolors.YL}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 2: Filling NaN values by year and SEA across group average')
    features_join = (
        features_join
        .fillna(features_join
                .groupby(['sea_unq'], as_index=False)
                .transform('median'), 
               )
    )
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Post step 2 NaN row count: {bcolors.GR}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPost step 2 NaN row %: {bcolors.GR}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPost step 2 NaN cell %: {bcolors.GR}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 3: Drop remaining NaN values\n')
    features_join = features_join.dropna(axis=0)
    print(f'Ending total row count: {bcolors.BL}{len(features_join)}{bcolors.RESET}',
          f'\nPost step 2 NaN cell %: {bcolors.GR}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}')
    
# The else section is a basic simple imputation
else: 
    # Store the geometry column separately
    geometry_col = features_join['geometry_y']
    # Remove the geometry column from the DataFrame
    features_join = features_join.drop(columns=['geometry_y'])
    features_join = features_join.set_index(['year', 'sea_unq'])
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    imputer.fit_transform(features_join)
    features_join[:] = imputer.transform(features_join)
    features_join = features_join.reset_index()
    # Add the geometry column back to the DataFrame
    features_join['geometry'] = geometry_col

Post step 1 NaN row count: 32888 
Post step 1 NaN row %: 99.87 
Post step 1 NaN cell %: 29.68 

Step 2: Filling NaN values by year and SEA across group average
Filled NAs for ['0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8', '0_9', '0_10', '0_11', '0_12']
Filled NAs for ['1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8', '1_9', '1_10', '1_11', '1_12']
Filled NAs for ['2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8', '2_9', '2_10', '2_11', '2_12']
Filled NAs for ['3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '3_9', '3_10', '3_11', '3_12']
Filled NAs for ['4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '4_9', '4_10', '4_11', '4_12']
Filled NAs for ['5_1', '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '5_9', '5_10', '5_11', '5_12']
Filled NAs for ['6_1', '6_2', '6_3', '6_4', '6_5', '6_6', '6_7', '6_8', '6_9', '6_10', '6_11', '6_12']
Filled NAs for ['7_1', '7_2', '7_3', '7_4', '7_5', '7_6', '7_7', '7_8', '7_9', '7_10', '7_11', '7_12']
Filled NAs for [

/tmp/ipykernel_2502043/63513881.py:38: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .transform('median')


Post step 2 NaN row count: 29826 
Post step 2 NaN row %: 90.57 
Post step 2 NaN cell %: 8.48 

Step 2: Filling NaN values by year and SEA across group average


/tmp/ipykernel_2502043/63513881.py:51: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .transform('median'),


Post step 2 NaN row count: 18341 
Post step 2 NaN row %: 55.70 
Post step 2 NaN cell %: 1.70 

Step 3: Drop remaining NaN values

Ending total row count: 14590 
Post step 2 NaN cell %: 0.00


In [31]:
features_join.iloc[:, 505:]

,41_10,41_11,41_12,42_1,42_2,42_3,42_4,42_5,42_6,42_7,...,log_maize,log_sweetpotatoes,log_groundnuts,log_soybeans,loss_ind,drought_loss_ind,flood_loss_ind,animal_loss_ind,pest_loss_ind,geometry_y
0,1.141906,0.968697,0.045290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
1,1.340785,1.161160,0.045290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
2,1.194933,1.089161,0.045290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.028847,2.333044,1.937424,1.586514,1.0,1.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
3,0.591394,0.325480,0.090580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
4,1.315837,1.325033,0.645732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32198,1.650558,1.814928,1.067077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.939201,9.367183,8.098897,7.336848,0.0,0.0,0.0,0.0,0.0,"POLYGON ((25.07771 -14.63920, 25.07732 -14.638..."
32199,1.646087,1.656592,1.067077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.939201,9.367183,8.098897,7.336848,0.0,0.0,0.0,0.0,0.0,"POLYGON ((25.07771 -14.63920, 25.07732 -14.638..."
32200,1.421446,1.498086,1.067077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.939201,9.367183,8.098897,7.336848,0.0,0.0,0.0,0.0,0.0,"POLYGON ((25.07771 -14.63920, 25.07732 -14.638..."
32201,1.187110,1.287694,1.067077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.939201,9.367183,8.098897,7.336848,0.0,0.0,0.0,0.0,0.0,"POLYGON ((25.07771 -14.63920, 25.07732 -14.638..."


In [32]:
features_new = features_join.copy()
features_new.drop(columns=['geometry_y'])
grouped_features = features_new.groupby(['year', 'sea_unq']).mean()
grouped_features.to_csv('/Users/hveirs/Python/mosaiks_repos/grouped_features.csv', index=True)

/tmp/ipykernel_2502043/1649593256.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_features = features_new.groupby(['year', 'sea_unq']).mean()


In [33]:
features_join.drop(columns=['geometry_y'])
grouped_features = features_new.groupby(['year', 'sea_unq']).mean()
grouped_features.to_csv('/Users/hveirs/Python/mosaiks_repos/features_join_imputed.csv', index=True)

/tmp/ipykernel_2502043/2638680127.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_features = features_new.groupby(['year', 'sea_unq']).mean()


In [1]:
# Now save this dataframe so you don't need to run this code again!
#features_join.to_file('/Users/hveirs/Python/mosaiks_repos/features_join_imputed.shp')
#features_join.to_feather('/Users/hveirs/Python/mosaiks_repos/features_join_imputed.feather')


NameError: name 'features_join' is not defined

# ---- Imputation Manual Testing Done ----

Now it would be ideal to get the above manual imputation parallel processed, but this is as far as I have done.
Below is my attempt at parallelizing the process:
# ---- Automated Imputation attempt ----

In [25]:
if impute_manual:
    ln_ft = len(features_join) # This saves the total length of the features_join dataframe 
    ln_na = len(features_join.dropna()) # This saves the length of the features_join dataframe without NA values 
    # (which is none since every row has at least 1 missing value)
    
    # This print statement simply helps us keep track of the number of rows 
    # and what we are currently on before starting the process.
    print(f'Starting total row count: {bcolors.BL}{ln_ft}{bcolors.RESET}',
          f'\nPre-Impute NaN row count: {bcolors.RD}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPre-Impute NaN row %: {bcolors.RD}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPre-Impute NaN cell %: {bcolors.RD}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 1: Filling NaN values by month and feature') 
    # We need to make a for loop so that the backfill method can iterate for each feature and not mess up between transitions of cols
    group_names = [f"{i}_{j}" for i in range(1000) for j in range(1, 13)]
    groups = [group_names[i:i+12] for i in range(0, len(group_names), 12)]

    # # Then use a for loop to basically make sure the columns are in the feature form. 
    # # We don't want to impute features based on survey columns.
    # for col in features_join.columns:
    #     col_str = str(col)
    #     if col_str in group_names:
    #         split_col = col_str.split("_")
    #         for group in groups:
    #             features_join[group] = features_join[group].fillna(method="bfill", axis=1)
    #             #print(f"Filled NAs for {group}")
    def fillna_for_group(df, group):
        df[group] = df[group].fillna(method="bfill", axis = "columns")
        print(f"Filled NAs for {group}")
        return df

    # Define the number of processes to use
    num_processes = multiprocessing.cpu_count() - 35

    # Split the group names into chunks for each process
    group_chunks = [group_names[i:i+num_processes] for i in range(0, len(group_names), num_processes)]

    # Create a process pool and map the function over each chunk of group names
    with multiprocessing.Pool(processes=num_processes) as pool:
        for group_chunk in group_chunks:
            results = [pool.apply_async(fillna_for_group, args=(features_join, group)) for group in group_chunk]
            output = [r.get() for r in results]
            
            
    ln_na = len(features_join.dropna())
    print(f'Post step 1 NaN row count: {bcolors.YL}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPost step 1 NaN row %: {bcolors.YL}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPost step 1 NaN cell %: {bcolors.YL}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 2: Filling NaN values by year and SEA across group average')
    features_join = (
        features_join
        .fillna(features_join
                .groupby(['year', 'sea_unq'], as_index=False) 
                .transform('median')
               )
    )
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Post step 2 NaN row count: {bcolors.YL}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPost step 2 NaN row %: {bcolors.YL}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPost step 2 NaN cell %: {bcolors.YL}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 2: Filling NaN values by year and SEA across group average')
    features_join = (
        features_join
        .fillna(features_join
                .groupby(['sea_unq'], as_index=False)
                .transform('median'), 
               )
    )
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Post step 2 NaN row count: {bcolors.GR}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPost step 2 NaN row %: {bcolors.GR}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPost step 2 NaN cell %: {bcolors.GR}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 3: Drop remaining NaN values\n')
    features_join = features_join.dropna(axis=0)
    print(f'Ending total row count: {bcolors.BL}{len(features_join)}{bcolors.RESET}',
          f'\nPost step 2 NaN cell %: {bcolors.GR}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}')
    
# The else section is a basic simple imputation
else: 
    # Store the geometry column separately
    geometry_col = features_join['geometry_y']
    # Remove the geometry column from the DataFrame
    features_join = features_join.drop(columns=['geometry_y'])
    features_join = features_join.set_index(['year', 'sea_unq'])
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    imputer.fit_transform(features_join)
    features_join[:] = imputer.transform(features_join)
    features_join = features_join.reset_index()
    # Add the geometry column back to the DataFrame
    features_join['geometry'] = geometry_col

TypeError: object of type 'NoneType' has no len()

In [ ]:
# Notes: Have to change the year, get an error rn. Also, check to make sure the number of cells is correct
# The if section of this chunk is the manual imputation method
# if impute_manual:
#     ln_ft = len(features_join) # This saves the total length of the features_join dataframe 
#     ln_na = len(features_join.dropna()) # This saves the length of the features_join dataframe without NA values 
#     # (which is none since every row has at least 1 missing value)
    
#     # This print statement simply helps us keep track of the number of rows 
#     # and what we are currently on before starting the process.
#     print(f'Starting total row count: {bcolors.BL}{ln_ft}{bcolors.RESET}',
#           f'\nPre-Impute NaN row count: {bcolors.RD}{ln_ft - ln_na}{bcolors.RESET}',
#           f'\nPre-Impute NaN row %: {bcolors.RD}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
#           f'\nPre-Impute NaN cell %: {bcolors.RD}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
#           f'\n\nStep 1: Filling NaN values by month, year, and SEA group average') 
#     # This is the line that takes the values and imputes the missing values based on the average.
#     features_join = ( 
#         features_join
#         .fillna(features_join # .fillna is saying to use the next statement to replace NA values with the resulting statement
#                 .groupby(['year', 'lat', 'lon'], as_index=False) # This groups the data based on year and unique SEA and we don't want to index based on the resulting df
#                 .transform('median'), # Then this means to take the average of the non-missing values (based on the groups above)
#                )
#     )
#     ln_ft = len(features_join)
#     ln_na = len(features_join.dropna())
#     print(f'Post step 1 NaN row count: {bcolors.YL}{ln_ft - ln_na}{bcolors.RESET}',
#           f'\nPost step 1 NaN row %: {bcolors.YL}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
#           f'\nPost step 1 NaN cell %: {bcolors.YL}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
#           f'\n\nStep 2: Filling NaN values by month and SEA across group average')
#     features_join = (
#         features_join
#         .fillna(features_join
#                 .groupby(['sea_unq'], as_index=False)
#                 .transform('mean'), 
#                )
#     )
#     ln_ft = len(features_join)
#     ln_na = len(features_join.dropna())
#     print(f'Post step 2 NaN row count: {bcolors.GR}{ln_ft - ln_na}{bcolors.RESET}',
#           f'\nPost step 2 NaN row %: {bcolors.GR}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
#           f'\nPost step 2 NaN cell %: {bcolors.GR}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
#           f'\n\nStep 3: Drop remaining NaN values\n')
#     features_join = features_join.dropna(axis=0)
#     print(f'Ending total row count: {bcolors.BL}{len(features_join)}{bcolors.RESET}')
    
# # The else section is a basic simple imputation
# else: 
#     # Store the geometry column separately
#     geometry_col = features_join['geometry_y']
#     # Remove the geometry column from the DataFrame
#     features_join = features_join.drop(columns=['geometry_y'])
#     features_join = features_join.set_index(['year', 'sea_unq'])
#     imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
#     imputer.fit_transform(features_join)
#     features_join[:] = imputer.transform(features_join)
#     features_join = features_join.reset_index()
#     # Add the geometry column back to the DataFrame
#     features_join['geometry'] = geometry_col

In [25]:
features_join

,index_left,lon,lat,year,0_1,0_2,0_3,0_4,0_5,0_6,...,log_maz,lg_swtp,lg_grnd,lg_sybn,loss_nd,drght__,fld_ls_,anml_l_,pst_ls_,geometry_y
0,46333.0,27.81466,-13.669357,2020.0,0.000000,0.000000,0.006237,0.000000,0.000000,0.000000,...,7.597093,8.205414,7.640386,7.10480,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
1,46288.0,27.80466,-13.659357,2020.0,0.000000,0.000000,0.006237,0.000000,0.000000,0.000000,...,7.597093,8.205414,7.640386,7.10480,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
2,46297.0,27.80466,-13.649357,2020.0,0.000000,0.000000,0.006237,0.000000,0.000000,0.000000,...,7.597093,8.205414,7.640386,7.10480,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
3,46334.0,27.81466,-13.669357,2021.0,0.000000,0.000000,0.006237,0.000000,0.000000,0.000000,...,7.667533,8.205414,7.855763,6.97709,1.0,1.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
4,46289.0,27.80466,-13.659357,2021.0,0.000000,0.000000,0.006237,0.000000,0.000000,0.000000,...,7.667533,8.205414,7.855763,6.97709,1.0,1.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24552,7307.0,23.24466,-16.279357,2020.0,0.001146,0.000854,0.000416,0.000353,0.000348,0.000426,...,5.648800,2.074326,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
24553,7571.0,23.26466,-16.279357,2020.0,0.005164,0.000706,0.000000,0.000000,0.000000,0.000172,...,5.648800,2.074326,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
24554,7197.0,23.23466,-16.279357,2020.0,0.000717,0.000353,0.000319,0.000559,0.000502,0.000581,...,5.648800,2.074326,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
24555,7430.0,23.25466,-16.269357,2020.0,0.004515,0.000035,0.000000,0.000000,0.000000,0.000167,...,5.648800,2.074326,6.830794,NaN,1.0,1.0,0.0,0.0,1.0,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."


### Test of IterativeImputer to try to Impute missing values more effectively

The Simple Imputer from sklearn is fine, but our ending number of points has been below 20% of the original dataframe (~2% using Carlo's features and Sitian's old joined ground data)

In [26]:
# This is a test to see if IterativeImputer would work with this data
# First going to make a copy of our joined features and just save the first 1000 rows or so.
features_test = features_join.copy().iloc[:200,:]
features_test

,index_left,lon,lat,year,0_1,0_2,0_3,0_4,0_5,0_6,...,log_maz,lg_swtp,lg_grnd,lg_sybn,loss_nd,drght__,fld_ls_,anml_l_,pst_ls_,geometry_y
0,46333.0,27.81466,-13.669357,2020.0,0.0,0.0,0.006237,0.00000,0.000000,0.000000,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
1,46288.0,27.80466,-13.659357,2020.0,0.0,0.0,0.006237,0.00000,0.000000,0.000000,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
2,46297.0,27.80466,-13.649357,2020.0,0.0,0.0,0.006237,0.00000,0.000000,0.000000,...,7.597093,8.205414,7.640386,7.104800,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
3,46334.0,27.81466,-13.669357,2021.0,0.0,0.0,0.006237,0.00000,0.000000,0.000000,...,7.667533,8.205414,7.855763,6.977090,1.0,1.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
4,46289.0,27.80466,-13.659357,2021.0,0.0,0.0,0.006237,0.00000,0.000000,0.000000,...,7.667533,8.205414,7.855763,6.977090,1.0,1.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,49979.0,28.43466,-13.509357,2021.0,0.0,NaN,0.000244,0.00000,0.000000,0.000000,...,7.904897,8.722160,8.296390,7.266261,0.0,0.0,0.0,0.0,0.0,"POLYGON ((28.39982 -13.51544, 28.40012 -13.514..."
196,49772.0,28.40466,-13.509357,2021.0,0.0,NaN,0.000063,0.00029,0.000081,0.000068,...,7.904897,8.722160,8.296390,7.266261,0.0,0.0,0.0,0.0,0.0,"POLYGON ((28.39982 -13.51544, 28.40012 -13.514..."
197,49781.0,28.40466,-13.489357,2021.0,0.0,NaN,0.000107,0.00000,0.000040,0.000028,...,7.904897,8.722160,8.296390,7.266261,0.0,0.0,0.0,0.0,0.0,"POLYGON ((28.39982 -13.51544, 28.40012 -13.514..."
198,49988.0,28.43466,-13.489357,2021.0,0.0,NaN,0.000010,0.00002,0.000000,0.000000,...,7.904897,8.722160,8.296390,7.266261,0.0,0.0,0.0,0.0,0.0,"POLYGON ((28.39982 -13.51544, 28.40012 -13.514..."


In [ ]:
# Load the Iterative Imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import random

In [27]:
# Try out the Iterative Imputer
# Set the seed
random.seed(987)
# Store the geometry column separately
#geometry_col = features_test['geometry_y']
# Remove the geometry column from the DataFrame
#features_test_new = features_test.drop(columns=['geometry_y'])
#features_test_new = features_test_new.set_index(['year', 'sea_unq'])
#imputer = IterativeImputer(missing_values=np.nan, max_iter=10)
#imputer.fit(features_test_new)
#imputer.fit_transform(features_test_new)
#features_test_new[:] = imputer.transform(features_test_new)
#features_test_new = features_test_new.reset_index()
# Add the geometry column back to the DataFrame
#features_test_new['geometry'] = geometry_col

### Save copy of processed features before sumarizing training features to district level

Duplicate the features dataframe at this stage so we can retain a copy of features at point resolution for all years available, which is `2013/2014/2016-2021`.

    - The start year is `2016` if the satellite selected is Sentinel 2 (due to the fact that Sentinel 2 launched in June of `2015`)
    - The start year is `2013` if the satellite selected is Landsat 8 and the month range selected was anything besides all months (due to the fact that Landsat 8 launched in February of `2013`)
    - The start year is `2014` if the satellite selected is Landsat 8 and the month range selected was all months
    
This duplicated dataframe we create in the following code is called `features_all_years`. The purpose for this dataframe comes into play after the model is trained; we will be able to plug in point-resolution features from _any and all_ years from this dataframe into the trained model and observe how the model predicts crop years across space and time. It would be interesting to plot these features for each year sequentially to show how the crop prediction landscape changes by year. These point-resolution features increase the spatial resolution of the ground-truth crop data we have for the years through 2018, because our ground-truth crop data is at a the coarser  district-resolution. Furthermore, these point-resolution features are the _only_ crop data we have for the years 2020-2021. The reason we lack data from 2019 is because the Zanbia Sattistics Agency has not yet released their Crop Forecast Survey data for that year. The reason we do not have data for 2020-2021 is because Covid-19 prevented any Crop Forecast Surveys from being conducted. 


After we create the dataframe `features_all_years`, we are free to further process the original features dataframe, `features_join`, in order to train the model with these features and their paired ground-truth crop yields. Processing this dataframe further requires us to subset the years to the start year through the years for which we have crop data: `2013/2014/2016-2019`. This dataframe is called `features_through_2019`. The reason we subset this dataframe is because we are training the model using _supervised_ machine learning, which means we are feeding it only features that have ground-truth crop data accosicated with them.

In [ ]:
features_all_years = features_join.copy()

# assign the geometry column to features_2014_2021 so it can serve 2 purposes:
# 1. plotting features sequentially by year
# 2. the entire dataframe can be fed into the model after the model is trained on only the summarized features for 2014-2018 and the associated crop data
# moving forward in the immediate sections, summarize the `features` dataframe to SEA level

### Summarise to administrative boundary level
Weighted by cropped area, or simple mean, depending on the selection at the top of this notebook for `weighted_avg`. 

In [ ]:
# check the order of the columns in the dataframe that will be summarized and then fed into the ridge regression in order to train the model
# we care about the order of columns specifically because in the following steps we assign only the feature columns to an object, so we need to know which 3 columns to omit by indexing
features_join.columns

The output above show that the 3 columns that are _not_ features are the first 2 columns `year` and `district`, and the last column, `crop_perc`.

In [ ]:
# check the shape to the dataframe as a sanity check
features_join.shape

In [ ]:
features_join.iloc[:, 2]

The output above shows the number of rows and columns in the dataframe, respectively. Recall that the number of rows represents the number of points for which we have features, and the number of columns is all features for all months selected plus the columns joined from the ground-truth data. There are _____ rows, meaning that is the amount of training points we have to feed into the model _before they are summarized to SEA level, so this number will shrink after we summarize to district level. There are 12060 columns, which will not change after we summarize the features to district level. The number of columns that we include in the features object in the next chunk will be this number minus the _____ non-feature columns.

In [28]:
# create object that contains only feature columns, rather than all columns that would include `district`, `year`, and `crop_perc`
# python index starts at 0, so here we specify to retain columns starting at 3 through every column besides the last column
# the columns we omit stay in the dataframe, because we assign the selected columns to an object, but the omitted columns are not included in the calculation in the next chunk
var_cols = features_join.columns[2:12001].values.tolist()

# call the object `var_cols` to check that it only includes feature columns, but do not view it in list format because it is more readable not as a list 
features_join.columns[2:12001]
# these are all the feature columns that will be fed into the `weighted_avg` calculation in the next chunk

Index(['lat', 'year', '0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8',
       ...
       '998_12', '999_1', '999_2', '999_3', '999_4', '999_5', '999_6', '999_7',
       '999_8', '999_9'],
      dtype='object', length=11999)

In [29]:
%%time
# Group by 'year' and 'sea_unq' and calculate the mean for the specified columns
grouped_features = features_join.groupby(['year', 'sea_unq']).mean()


CPU times: user 2.25 s, sys: 28.6 ms, total: 2.28 s
Wall time: 2.28 s


<timed exec>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.


Now that the features have been summarized to district and year, there are fewer rows. The dataframe we were working with before this step,  `features_through_2018`, had 13866 rows that represented points. Now we have 216 rows, as shown by the following output. Notice we still have all 12003 columns. 

In [30]:
grouped_features.iloc[:, 12001:]

999_11    999_12   ttl_r_p_  ttl_r_h_  ttl_r_l_   ttl_hr_  \
year   sea_unq                                                                
2016.0 1        0.041070  1.000000    76.0000   71.0000    5.0000    4400.0   
       2        0.004529  0.018737   131.0000  124.0000    7.0000    3150.0   
       7        0.001009  0.004981  1016.0000  607.0000  409.0000    8730.0   
       9        0.002619  0.013608   652.0000  462.0000  190.0000    7930.0   
       10       0.005074  0.011449   545.0000  410.0000  135.0000   19975.0   
...                  ...       ...        ...       ...       ...       ...   
2021.0 388      0.000569       NaN    42.3725   42.1725    0.2000   96551.0   
       389      0.001119  0.001536    20.2500   16.2500    4.0000   42600.0   
       390      0.002530  0.005225    24.4350   21.7175    2.7175   52865.0   
       391      0.020975  0.006625    62.3500   51.3000   11.0500  147560.0   
       392      0.001969  0.003385    41.6625   32.4100    9.2525   88640.0   

                    yld_kgh   frc_r_h   frc_r_l  ar_lst_  ...   prop_mx  \
year   sea_unq                                            ...             
2016.0 1          57.894737  0.934211  0.065789      0.0  ...  0.000000   
       2          24.045802  0.946565  0.053435      0.0  ...  0.000000   
       7           8.592520  0.597441  0.402559      0.0  ...  0.181102   
       9          12.162577  0.708589  0.291411      0.0  ...  0.069018   
       10         36.651376  0.752294  0.247706      0.0  ...  0.000000   
...                     ...       ...       ...      ...  ...       ...   
2021.0 388      2278.624108  0.995280  0.004720      0.0  ...  0.000000   
       389      2103.703704  0.802469  0.197531      0.0  ...  0.000000   
       390      2163.494987  0.888787  0.111213      0.0  ...  0.000000   
       391      2366.639936  0.822775  0.177225      0.0  ...  0.000000   
       392      2127.572757  0.777918  0.222082      0.0  ...  0.000000   

                 log_maz   lg_swtp   lg_grnd   lg_sybn  loss_nd  drght__  \
year   sea_unq                                                             
2016.0 1        4.058626  8.205414  7.748075  7.040945      0.0      0.0   
       2        3.179960       NaN  7.832302  7.205323      0.0      0.0   
       7        3.387211  0.689155  8.031431  7.113191      1.0      1.0   
       9        2.703935  8.776231 -1.408767  7.174615      1.0      0.0   
       10       3.714757  2.525729  3.354421  7.115509      1.0      0.0   
...                  ...       ...       ...       ...      ...      ...   
2021.0 388      7.939201  9.367183  8.098897  7.336848      0.0      0.0   
       389      7.844633       NaN  8.048788       NaN      1.0      1.0   
       390      7.792168  7.863267  8.154788       NaN      1.0      0.0   
       391      7.652396       NaN  8.065208       NaN      1.0      0.0   
       392      7.641359  2.074326  7.960925       NaN      1.0      0.0   

                fld_ls_  anml_l_  pst_ls_  
year   sea_unq                             
2016.0 1            0.0      0.0      0.0  
       2            0.0      0.0      0.0  
       7            0.0      0.0      0.0  
       9            0.0      0.0      0.0  
       10           0.0      0.0      0.0  
...                 ...      ...      ...  
2021.0 388          0.0      0.0      0.0  
       389          0.0      0.0      0.0  
       390          0.0      0.0      0.0  
       391          1.0      0.0      0.0  
       392          0.0      0.0      0.0  

[1015 rows x 41 columns]

## Model

### Define `x`'s and `y`'s that will be a part of training the model

Since our independent variable is the features, these are the `x`'s. Our dependent variable is the crop yield in metric tonnes per hectare planted, so that will be the `y`'s.

In [31]:
# Separate features (X) and target variables (y)
X = grouped_features.iloc[:, 2:12000]
y = grouped_features["log_maize"]

In [32]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Split into train and test sets

This step is executed right before training the model so we can train on 80% of the data and preserve 20% for testing.

In [33]:
print("Number of total points: ", len(X), "\n", 
      "Number of training points: ", len(X_train), "\n",
      "Number of testing points: ", len(X_test), sep = "")

Number of total points: 1015
Number of training points: 812
Number of testing points: 203


### Train model using cross-validated ridge regression

Please see the documentation for the function that executes this regression [here.](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)

In [34]:
# Create a pipeline with normalization
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RidgeCV(cv = 5, alphas = np.logspace(-8, 8, base = 10, num = 17)))
])


In [35]:
# Train the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)

ValueError: 
All the 85 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
85 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/hveirs/.conda/envs/mosaiks/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/hveirs/.conda/envs/mosaiks/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py", line 1126, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/hveirs/.conda/envs/mosaiks/lib/python3.11/site-packages/sklearn/base.py", line 565, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hveirs/.conda/envs/mosaiks/lib/python3.11/site-packag

In [ ]:
y_pred

In [ ]:
# Initialize lists to store the predictions, RMSEs, and R-squared values
predictions = []
rmse_list = []
r2_list = []

# Loop through the target variables (columns)
for i in range(y_test.shape[1]):
    # Extract the true values and predictions for the current target variable
    y_test_i = y_test.iloc[:, i]
    y_pred_i = y_pred[:, i]
    
    # Compute the RMSE and R-squared
    rmse_i = np.sqrt(mean_squared_error(y_test_i, y_pred_i))
    r2_i = r2_score(y_test_i, y_pred_i)
    
    # Append the results to the corresponding lists
    predictions.append(y_pred_i)
    rmse_list.append(rmse_i)
    r2_list.append(r2_i)

# Print the RMSE and R-squared values for each target variable

for i, (column_name, rmse_i, r2_i) in enumerate(zip(y_test.columns, rmse_list, r2_list), start=1):
    print(f"{column_name}: RMSE = {rmse_i:.4f}, R-squared = {r2_i:.4f}")

### Validation set $R^2$ performance

In [ ]:
print(f"Validation R2 performance: {ridge_cv_random.best_score_:0.2f}")

### Train set

In [ ]:
y_pred = np.maximum(ridge_cv_random.predict(x_train), 0)
r2_train = r2_score(y_train, y_pred)

fig, ax = plt.subplots(ncols=1)
plt.scatter(y_pred, y_train, alpha=1, s=4)
plt.xlabel("Predicted", fontsize=15, x = .3)
plt.ylabel("Ground Truth", fontsize=15)
plt.suptitle(r"$\log_{10}(1 + Crop Yield)$", fontsize=20, y=1.02)
plt.title((f"Model applied to train data n = {len(x_train)}, R$^2$ = {r2_train:0.2f}"),
          fontsize=12, y=1.01)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

ax.axline([0, 0], [1, 1], c = "k")

plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)


# plt.savefig(f'images/{feature_file_name}_train_data.jpg', dpi=300)
plt.show()
plt.close()
# the model is plotted with a black 45 degree line that serves as a reference of what a perfect correlation would look like
# deviation of the line indicates that there is not a perfect correlation

In [ ]:
print(f"Training R^2 = {r2_train:0.2f}\nPearsons r = {pearsonr(y_pred, y_train)[0]:0.2f}") 

In [ ]:
# Pearson r^2
pearsonr(y_pred, y_train)[0] ** 2

In [ ]:
# alternative way to calculate Training R^2
ridge_cv_random.score(x_train, y_train)

### Test set

In [ ]:
y_pred = np.maximum(ridge_cv_random.predict(x_test), 0)
r2_test = r2_score(y_test, y_pred)

plt.figure()
plt.scatter(y_pred, y_test, alpha=1, s=4)
plt.xlabel("Predicted", fontsize=15)
plt.ylabel("Ground Truth", fontsize=15)
plt.suptitle(r"$\log_{10}(1 + Crop Yield)$", fontsize=20, y=1.02)
plt.title(f"Model applied to test data n = {len(x_test)}, R$^2$ = {r2_test:0.2f}",
          fontsize=12, y=1)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

ax.axline([0, 0], [.75, .75], c = "k")

plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)

# plt.savefig(f'images/{feature_file_name}_test_data.jpg', dpi=300)
plt.show()
plt.close()

In [ ]:
print(f"Testing set R^2 = {r2_test:0.2f}")
print(f"Testing set pearsons R = {pearsonr(y_pred, y_test)[0]:0.2f}")

Summary of both train and test data sets

In [ ]:
y_pred = np.maximum(ridge_cv_random.predict(x_all), 0)

fig, ax = plt.subplots(figsize=(7, 7))
ax.axline([0, 0], [.75, .75], c = "k")
plt.scatter(y_pred, y_all, alpha=.9, s=15)
plt.xlabel("Predicted", fontsize=15)
plt.ylabel("Observed", fontsize=15)
plt.text(
    0, .8, fontsize=15, fontweight="bold",
    s=f"R$^2$={r2_train:0.2f} - Train set",
)
plt.text(
    0, .75, fontsize=15, fontweight="bold",
    s=f"R$^2$={ridge_cv_random.best_score_:0.2f} - Validation set",
)
plt.text(
    0, .7, fontsize=15, fontweight="bold",
    s=f"R$^2$={r2_test:0.2f} - Test set",
)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)

# plt.savefig(f'images/{feature_file_name}_all_data.jpg', dpi=300)
plt.show()
plt.close()

### Use the trained model to predict crop yields over all years from 1km grid-cell resolution features 

Recall that after we executed imputation on all feature years in the dataframe `features`, we copied the dataframe and named it `features_all_years`. Now we can plug that into the model to visualize how our model performs over time.

In [ ]:
# recall the object we created earlier, before we split the features by year into those that would train the model 
# and those that would be fed into the trained model to predict crop yields
# in years for which we do not have crop data
features_all_years.head(3)

In the following chunk, we drop certain columns from `features_all_years` because we only need to feed the feature data into the model to generate predictions. Using the argument `axis = 1`, we specify that we are dropping columns rather than rows. 

In [ ]:
x_all = features_all_years.drop([
    'year', 
    'geometry',
    'district',
    'crop_perc'
], axis = 1)

In the following chunk, we execute the model on the features from the dataframe `features_all_years`. The crop yield predictions for each row populate a new column in the dataframe.

The model is run inside the `np.maximum()` function because if we run it without being wrapped inside function, some crop predictions are negative values, but we need them all to be positive because conceptually crop yields cannot be negative.

In [ ]:
features_all_years['yield_prediction'] = np.maximum(ridge_cv_random.predict(x_all), 0)

In [ ]:
# check out the dataframe with the new column of predictions
features_all_years.head(3)

The dataframe is already a geodataframe, so we do not have to convert it to one before mapping predictions. However, we do need to replace all the zero value crop percentage areas with `NA`. We do this by applying the `mask()` function. This function is similar to an if-else statement. If the value of the `crop_perc` is equal to 0, that value is replaced by the value of the second argument, which is `NA`. If the value of `crop_prec` is _not_ equal to zero, we retain the current value. The argument `inplace = True` executes this replacement in the same cell. 

In [ ]:
features_all_years['yield_prediction'].mask(features_all_years['crop_perc']==0, np.nan, inplace=True)

Recall that this dataframe has a geometry column, with latitude and longitude together. In order to map the predicted features, we separate this geometry column into separate `lon` and `lat` columns. 

In [ ]:
# extract the longitude and latitude from the geometry column, and make then into independent columns
features_all_years['lon'], features_all_years['lat'] = features_all_years.geometry.x, features_all_years.geometry.y

Plot the predicted features for each year:

In [ ]:
def scatter(x, y, c, **kwargs):
    plt.scatter(x, y, c=c, s = 1.25)
sns.color_palette("viridis", as_cmap=True)
g = sns.FacetGrid(
    features_all_years, 
    col="year", 
    col_wrap = 4, 
    height=5, 
    aspect=1
)
g.map(scatter, "lon", "lat", "yield_prediction")
g.set_axis_labels(r"Yield Prediction")
# save the figure and name the file so that it represents the model parameters that created the predictions
# plt.savefig(f'images/{feature_file_name}_all_predictions.jpg', dpi=300)

Plot the model's predicted features summarized to district level. In this visualization, we choose a specific year to examine rather than visualizing all years in one figure. Visualizing the the features summarized to district level is interesting because the crop data resolution provided by Zambia Statistics Agency is at the district level, and therefore it is easier to compare our model results to those ground-truth values when they are summarized to district level as well. Furthermore, our model's crop predictions for the years 2020 and 2021 might be more valuable when summarized to district level if Zambian governments, policy-makers, farmers, and researchers wish to use this data to determine crop imports, exports, and storage according to district summaries. 

In [ ]:
features_all_years_summary = (
    features_all_years
    .groupby(['district',"year"], as_index = False)['yield_prediction']
    .mean()
    .set_index('district')
)

In [ ]:
# join Zambia's shapefile to the summarized features to map the districts
# reset the index so it is a properly formatted dataframe
features_all_years_summary = features_all_years_summary.join(country_shp).reset_index()

Now that the geometries have been converted to districts from points, the geomatries are now polygons. There is still a row for each district for each year.

In order to change the year visualized, simply change the year in the following code and re-run the chunk.

In [ ]:
features_all_years_summary[features_all_years_summary.year == 2020].plot(column = "yield_prediction")

Plot a boxplot for each year to visualize the range and quantile distribution of each year's crop predictions, summarized to district level. This enables us to identify years with exceptional disparities between the predicted yields by district. It also allows us to identify years that have many outliers.

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x="year", y="yield_prediction", data = features_all_years_summary)
plt.xlabel("Year", fontsize=15)
plt.ylabel("Predicted Yield", fontsize=15)

Visualize the total crop yield predictions by year. This bar chart shows the sum of all the district crop yields.

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x="year", y="yield_prediction", data = features_all_years_summary, estimator = sum)

## Yield and Residual Plots

Create a dataframe of residuals called `residuals_df` from the `features_summary` dataframe. Note that we are _not_ using the predicted crop yields for _all_ years for these residuals, but rather the ground-truth crop yields for just the years through 2018.

The residuals give us an idea of the amount of uncertianty that is present in our model. By demeaning the residuals over space, we are able to remove the uncertainty over space and better determine our model performance over time and our uncertainty over time.

In [ ]:
x_all = features_summary.drop(drop_cols, axis = 1)

# create empty dataframe to then populate with columns
residual_df = pd.DataFrame()

residual_df["yield_mt"] = features_summary.yield_mt.to_numpy()
residual_df["log_yield"] = np.log10(features_summary.yield_mt.to_numpy() + 1)
residual_df["prediction"] = np.maximum(ridge_cv_random.predict(x_all), 0)
residual_df["residual"] = residual_df["log_yield"] - residual_df["prediction"]
residual_df["year"] = features_summary.year
residual_df["district"] = features_summary.district
# join the district geometries
residual_df = residual_df.join(country_shp, how = "left", on = "district")

# demean by location so we can analyze the data over time
residual_df["district_yield_mean"] = residual_df.groupby('district')['log_yield'].transform('mean')
residual_df["district_prediction_mean"] = residual_df.groupby('district')['prediction'].transform('mean')
residual_df["demean_yield"] = residual_df["log_yield"] - residual_df["district_yield_mean"]
residual_df["demean_prediction"] = residual_df["prediction"] - residual_df["district_prediction_mean"]
residual_gdf = geopandas.GeoDataFrame(residual_df)

residual_gdf.head(3)

Visualize the residuals for the ground truth crop yields through 2018 with a boxplot.

In [ ]:
plt.figure(figsize=(6, 5))
sns.boxplot(x="year", y="log_yield", data=residual_df)
plt.xlabel("Year", fontsize=15)
plt.ylabel("Log Yield", fontsize=15)

Visualize the residuals as a sum by year with a bar plot.

In [ ]:
plt.figure(figsize=(6, 5))
sns.barplot(x="year", y="log_yield", data=residual_df, estimator = sum)

Visualize the crop yield residuals by year as a histogram to determine how they are distributed.

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.histplot, "yield_mt", bins = 20)
g.set_axis_labels("Yield (MT)")

Visualize the log-transformed crop yield residuals by year as a histogram to compare how they are distributed after the transformation.

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.histplot, "log_yield", bins = 20)
g.set_axis_labels(r"$\log_{10}(1 + Crop Yield)$")

#### Crop prediction histogram

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.histplot, "prediction", bins = 20)
g.set_axis_labels(r"Crop yield predictions")

#### Residual histogram

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.histplot, "residual", bins = 20)
g.set_axis_labels(r"Residuals")

In [ ]:
residual_gdf.residual.min()

In [ ]:
residual_gdf.residual.max()

#### Log crop yield vs residuals

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.scatterplot, "log_yield", "residual")
g.set_axis_labels(r"$\log_{10}(1 + Crop Yield)$")

#### District residuals 

In [ ]:
if satellite == 'landsat-8-c2-l2':
    fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(13, 5))
    ax1 = (residual_gdf[residual_gdf.year == 2014]
           .plot(ax = ax1, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
           .set_title("2014 Residuals"))
    ax2 = (residual_gdf[residual_gdf.year == 2015]
           .plot(ax = ax2, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
           .set_title("2015 Residuals"))
else:
    pass
fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
ax1 = (residual_gdf[residual_gdf.year == 2016]
       .plot(ax = ax1, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
       .set_title("2016 Residuals"))
ax2 = (residual_gdf[residual_gdf.year == 2017]
       .plot(ax = ax2, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
       .set_title("2017 Residuals"))
ax3 = (residual_gdf[residual_gdf.year == 2018]
       .plot(ax = ax3, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
       .set_title("2018 Residuals"))

caption = "A positive value is an underestimated prediction (the prediction is lower than the actual yield), a negative value is an over estimated prediction"
plt.figtext(0.5, 0.01, caption, wrap=True, horizontalalignment='center', fontsize=12)


#### Difference from the mean

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.scatterplot, "demean_yield", "demean_prediction")
g.set_axis_labels('Difference from Yield Mean', 'Difference from Prediction Mean')

In [ ]:
fig, ax = plt.subplots(figsize= (6, 5))
ax.axline([-.2, -.2], [.2, .2], c = "k")
plt.scatter(residual_gdf.demean_yield, residual_gdf.demean_prediction)
plt.title("Demeaned truth and predictions by district")
plt.xlabel('Difference from Yield Mean')
plt.ylabel('Difference from Predictions Mean')
r_squared = r2_score(residual_gdf["demean_yield"], residual_gdf["demean_prediction"])
plt.text(
    -0.2,
    .18,
    s=f"Demeaned R$^2$ = {r_squared:0.2f}",
    fontsize=15,
    fontweight="bold",
)
plt.savefig(f'images/{feature_file_name}_demean.jpg', dpi=300)

In [ ]:
for yr in range(year_start+1, 2018):
    r_squared = r2_score(residual_gdf[residual_gdf.year == yr]["demean_yield"], residual_gdf[residual_gdf.year == yr]["demean_prediction"])
    pearson_r = pearsonr(residual_gdf[residual_gdf.year == yr]["demean_yield"], residual_gdf[residual_gdf.year == yr]["demean_prediction"])
    
    print(yr, f"    R^2: {r_squared:.2f}\n",
          f"Pearson's r: {pearson_r[0]:.2f}\n", 
          sep = "")
    
r_squared = r2_score(residual_gdf["demean_yield"], residual_gdf["demean_prediction"])
pearson_r = pearsonr(residual_gdf["demean_yield"], residual_gdf["demean_prediction"])
print(f"All     R^2: {r_squared:.2f}\n",
      f"Pearson's r: {pearson_r[0]:.2f}", sep = "")

In [ ]:
r2 = round(pearson_r[0] ** 2, 2)
r2

#### Join residuals to the features for _all_ years to visualize the residuals of the features before they were summarized to district level.

In [ ]:
complete_df = (
    features_all_years_summary
    .set_index(['district', 'year'])
    .join(residual_df
          .drop('geometry', axis = 1)
          .set_index(['district', 'year'])
         )
    .reset_index()
)

complete_df.head(3)

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))
tidy = complete_df.melt(id_vars='year').rename(columns=str.title)
tidy = tidy[tidy.Variable.isin(['yield_prediction', 'log_yield'])]
sns.barplot(x='Year', y='Value', hue='Variable', data=tidy, ax=ax1, ci = None)
sns.despine(fig)

h, l = ax1.get_legend_handles_labels()
ax1.legend(h, ['Predicted Yield', 'Observed Yield'],loc='lower left')

plt.savefig(f'images/{feature_file_name}_yield_pred.jpg', dpi=300)

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x="year", y="yield_prediction", data=complete_df, estimator = sum)

### Congratulations on completing this analysis!